# 06: Autocorrelation & Stationarity Basics

このノートでは、リターンとリターン二乗の ACF を比較し、
さらにローリング平均・ローリング分散を見て定常性の直感を掴みます。

自己相関の式:

$$\rho_k = \frac{\sum_{t=k+1}^{T}(x_t-\bar{x})(x_{t-k}-\bar{x})}{\sum_{t=1}^{T}(x_t-\bar{x})^2}$$

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf

SYMBOL = os.getenv("SYMBOL", "QQQ")
PERIOD = "5y"
INTERVAL = "1d"

plt.style.use("seaborn-v0_8")

In [ ]:
df = yf.download(SYMBOL, period=PERIOD, interval=INTERVAL, auto_adjust=True, progress=False)
if isinstance(df.columns, pd.MultiIndex):
    close = df[("Close", SYMBOL)]
else:
    close = df["Close"]

data = pd.DataFrame({"close": close}).dropna()
data["ret"] = np.log(data["close"] / data["close"].shift(1))
data = data.dropna()

# Rolling stats for stationarity intuition
data["price_roll_mean"] = data["close"].rolling(60).mean()
data["price_roll_var"] = data["close"].rolling(60).var(ddof=0)
data["ret_roll_mean"] = data["ret"].rolling(60).mean()
data["ret_roll_var"] = data["ret"].rolling(60).var(ddof=0)

data.tail()

## ACF の手実装（statsmodels 不要）

In [ ]:
def simple_acf(series: pd.Series, max_lag: int = 40) -> pd.Series:
    x = pd.Series(series).dropna().to_numpy(dtype=float)
    x = x - x.mean()
    denom = np.dot(x, x)
    if denom == 0:
        return pd.Series([np.nan] * max_lag, index=range(1, max_lag + 1))

    out = []
    for lag in range(1, max_lag + 1):
        if lag >= len(x):
            out.append(np.nan)
        else:
            out.append(np.dot(x[lag:], x[:-lag]) / denom)
    return pd.Series(out, index=range(1, max_lag + 1))

acf_ret = simple_acf(data["ret"], max_lag=40)
acf_ret_sq = simple_acf(data["ret"] ** 2, max_lag=40)

print("ACF(ret) first 5 lags:")
print(acf_ret.head())
print("\nACF(ret^2) first 5 lags:")
print(acf_ret_sq.head())

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(13, 12))

axes[0].plot(data.index, data["close"], label="Price", color="tab:blue")
axes[0].plot(data.index, data["price_roll_mean"], label="Price rolling mean(60)", color="tab:orange")
axes[0].set_title(f"{SYMBOL} Price and Rolling Mean")
axes[0].legend()

axes[1].plot(data.index, data["price_roll_var"], label="Price rolling var(60)")
axes[1].plot(data.index, data["ret_roll_var"], label="Return rolling var(60)")
axes[1].set_title("Rolling Variance: Price vs Returns")
axes[1].legend()

lags = np.arange(1, len(acf_ret) + 1)
axes[2].stem(lags - 0.1, acf_ret.values, linefmt="C0-", markerfmt="C0o", basefmt="k-", label="ACF(ret)")
axes[2].stem(lags + 0.1, acf_ret_sq.values, linefmt="C1-", markerfmt="C1s", basefmt="k-", label="ACF(ret^2)")
axes[2].set_title("ACF Comparison")
axes[2].set_xlabel("Lag")
axes[2].legend()

plt.tight_layout()
plt.show()

## Optional: ADF test (statsmodels がある場合のみ)

In [ ]:
try:
    from statsmodels.tsa.stattools import adfuller

    adf_price = adfuller(data["close"].dropna())
    adf_ret = adfuller(data["ret"].dropna())

    print("ADF p-value (price):", adf_price[1])
    print("ADF p-value (return):", adf_ret[1])
    print("一般には price の方が非定常、return は定常に近い結果になりやすい。")
except Exception:
    print("statsmodels が未インストールのため ADF test をスキップしました。")
    print("必要なら: pip install statsmodels")

## What you should notice

- 価格系列のローリング平均・分散は時間とともに変化しやすく、非定常の直感につながる。
- リターン系列は価格より平均が安定し、扱いやすい。
- ACF(ret) は小さいことが多い一方、ACF(ret^2) は残りやすい。
- 方向よりも“変動の大きさ”に予測可能性が残る、という実務上の重要な示唆が得られる。